In [29]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from google.colab import files

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import warnings
warnings.filterwarnings('ignore')

In [4]:
#importing CSV uploaded to drive
id = "1M5IPR96R9efi5cb2n8LxpXIXyx_F0x3v"
print("https://drive.google.com/uc?export=download&id=" + id)
!wget "https://drive.google.com/uc?export=download&id=1M5IPR96R9efi5cb2n8LxpXIXyx_F0x3v" -O Online_Retail.csv

https://drive.google.com/uc?export=download&id=1M5IPR96R9efi5cb2n8LxpXIXyx_F0x3v
--2025-12-01 05:25:21--  https://drive.google.com/uc?export=download&id=1M5IPR96R9efi5cb2n8LxpXIXyx_F0x3v
Resolving drive.google.com (drive.google.com)... 64.233.181.102, 64.233.181.101, 64.233.181.100, ...
Connecting to drive.google.com (drive.google.com)|64.233.181.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1M5IPR96R9efi5cb2n8LxpXIXyx_F0x3v&export=download [following]
--2025-12-01 05:25:21--  https://drive.usercontent.google.com/download?id=1M5IPR96R9efi5cb2n8LxpXIXyx_F0x3v&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.195.132, 2607:f8b0:4001:c11::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.195.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45005763 (43M) [application/octet-stream]
Saving to:

In [6]:
df = pd.read_csv("Online_Retail.csv")
print(df.shape)
df.head()

(541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/10 8:26,3.39,17850.0,United Kingdom


In [7]:
# Try to find unique orders - invoices
# Unique customers

print('Number of Unique Invoice numbers: {cnt}'.format(cnt=df.InvoiceNo.nunique()))
print('Number of Unique Customer IDs: {cnt}'.format(cnt=df.CustomerID.nunique()))

Number of Unique Invoice numbers: 25900
Number of Unique Customer IDs: 4372


In [9]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [11]:
df = df[df['Quantity'] >= 0]
print(df.shape)

(531285, 8)


In [13]:
df.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,592
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,133361
Country,0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 531285 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    531285 non-null  object 
 1   StockCode    531285 non-null  object 
 2   Description  530693 non-null  object 
 3   Quantity     531285 non-null  int64  
 4   InvoiceDate  531285 non-null  object 
 5   UnitPrice    531285 non-null  float64
 6   CustomerID   397924 non-null  float64
 7   Country      531285 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 36.5+ MB


In [ ]:
 # Invoices that start fronm

In [20]:
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
# df['CustomerID'] = df['CustomerID'].astype('str')

In [22]:
print(df['Country'].nunique())
df['Country'].value_counts()

38


,count
Country,
United Kingdom,486286
Germany,9042
France,8408
EIRE,7894
Spain,2485
Netherlands,2363
Belgium,2031
Switzerland,1967
Portugal,1501


In [ ]:
# Preparing data for Apriori and Association Rules

In [26]:
# Filter data for the top country
data = df[df['Country'] == "United Kingdom"]
data = data.groupby(['InvoiceNo', 'Description'])['Quantity'].sum()
data = data.unstack().reset_index().fillna(0)
data = data.set_index('InvoiceNo')
data.head(3)

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,...,returned,taig adjust,test,to push order througha s stock was,website fixed,wrongly coded 20713,wrongly coded 23343,wrongly marked,wrongly marked 23343,wrongly sold (22719) barcode
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
#Replace quantities with Binary values
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

data = data.applymap(encode_units)
data.head(3)

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,...,returned,taig adjust,test,to push order througha s stock was,website fixed,wrongly coded 20713,wrongly coded 23343,wrongly marked,wrongly marked 23343,wrongly sold (22719) barcode
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# Filtering invoices whichn contains at least 2 different products are present

data = data[(data > 0).sum(axis=1) >= 2]
print(data.shape)
data.head()


(16539, 4058)


Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,...,returned,taig adjust,test,to push order througha s stock was,website fixed,wrongly coded 20713,wrongly coded 23343,wrongly marked,wrongly marked 23343,wrongly sold (22719) barcode
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536372,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# Apriori Algorithm

frequent_itemsets = apriori(data, min_support = 0.03, use_colnames = True)
frequent_itemsets.head(3)

,support,itemsets
0,0.051696,(6 RIBBONS RUSTIC CHARM)
1,0.035129,(60 CAKE CASES VINTAGE CHRISTMAS)
2,0.045529,(60 TEATIME FAIRY CAKE CASES)


In [32]:
frequent_itemsets = frequent_itemsets.sort_values('support', ascending = False).reset_index(drop = True)
frequent_itemsets.head(3)

,support,itemsets
0,0.129875,(WHITE HANGING HEART T-LIGHT HOLDER)
1,0.116331,(JUMBO BAG RED RETROSPOT)
2,0.100671,(REGENCY CAKESTAND 3 TIER)


In [33]:
frequent_itemsets.tail()

,support,itemsets
172,0.030473,(CHRISTMAS CRAFT LITTLE FRIENDS)
173,0.030473,(CREAM HEART CARD HOLDER)
174,0.030413,"(LUNCH BAG BLACK SKULL., LUNCH BAG CARS BLUE)"
175,0.030292,"(LUNCH BAG RED RETROSPOT, LUNCH BAG SPACEBOY D..."
176,0.030171,"(LUNCH BAG BLACK SKULL., LUNCH BAG SUKI DESIGN )"


In [34]:
# Finding Association Rules

association_rules = association_rules(frequent_itemsets, metric='lift',
                                      min_threshold=1)

association_rules  = association_rules.sort_values('lift', ascending=False).reset_index(drop=True)
association_rules.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.042264,0.056473,0.034887,0.825465,14.617093,1.0,0.032500,5.405948,0.972697,0.546402,0.815019,0.721619
1,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.056473,0.042264,0.034887,0.617773,14.617093,1.0,0.032500,2.505674,0.987345,0.546402,0.600906,0.721619
2,(PINK REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.042264,0.057682,0.033013,0.781116,13.541798,1.0,0.030575,4.305101,0.967025,0.493225,0.767717,0.676721
3,(ROSES REGENCY TEACUP AND SAUCER ),(PINK REGENCY TEACUP AND SAUCER),0.057682,0.042264,0.033013,0.572327,13.541798,1.0,0.030575,2.239413,0.982847,0.493225,0.553454,0.676721
4,(GARDENERS KNEELING PAD CUP OF TEA ),(GARDENERS KNEELING PAD KEEP CALM ),0.045287,0.054235,0.032711,0.722296,13.317793,1.0,0.030254,3.405662,0.968786,0.489593,0.706371,0.662709
